In [1]:
!pip install scipy
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize # Import the minimize function



In [2]:
msft = yf.download(tickers='MSFT', period='1mo')
lly = yf.download(tickers='LLY', period='1mo')
cvx = yf.download(tickers='CVX', period='1mo')
aapl = yf.download(tickers='AAPL', period='1mo')
bhp = yf.download(tickers='BHP', period='1mo')


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
msft['Return'] = msft['Adj Close'].pct_change()
lly['Return'] = lly['Adj Close'].pct_change()
cvx['Return'] = cvx['Adj Close'].pct_change()
aapl['Return'] = aapl['Adj Close'].pct_change()
bhp['Return'] = bhp['Adj Close'].pct_change()

returns = pd.DataFrame({
    'MSFT': msft['Return'],
    'LLY': lly['Return'],
    'CVX': cvx['Return'],
    'AAPL': aapl['Return'],
    'BHP': bhp['Return']
})

# Drop any rows with NaN values
returns.dropna(inplace=True)

# Calculate the variance-covariance matrix
C = returns.cov()

print(C)


          MSFT       LLY       CVX      AAPL       BHP
MSFT  0.000076  0.000019 -0.000039  0.000108 -0.000003
LLY   0.000019  0.000092 -0.000013  0.000012 -0.000033
CVX  -0.000039 -0.000013  0.000115 -0.000078  0.000005
AAPL  0.000108  0.000012 -0.000078  0.000490 -0.000104
BHP  -0.000003 -0.000033  0.000005 -0.000104  0.000182


In [4]:
#METHOD- 1 IMPLEMENTATION

w = np.ones(5)/5   #initial guess for weight matrix
C = np.array(C)
def obj_function(w, C):
    P = C @ w
    MCR = P/np.sqrt(w.T @ C @ w)
    f = 0.0
    for i in range(len(w)):
        for j in range(len(w)):
            f += (w[i] * MCR[i] - w[j] * MCR[j]) ** 2
    return f

def objective(w):
    return obj_function(w, C)

# Constraints
constraints = (
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # Equality constraint: sum(w) = 1
    {'type': 'ineq', 'fun': lambda w: w}             # Inequality constraint: w >= 0
)

# Bounds
bounds = [(0, 1) for _ in range(C.shape[1])]



# Optimize
result = minimize(objective, w, method='SLSQP', constraints=constraints, bounds=bounds, options={'disp': True})

# Optimal weights
optimal_weights = result.x

print("Optimal weights:", optimal_weights)
print("Objective function value at optimal weights:", result.fun)
print("Success:", result.success)
print("Message:", result.message)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.892108513323724e-06
            Iterations: 5
            Function evaluations: 30
            Gradient evaluations: 5
Optimal weights: [0.20411886 0.21771817 0.22830818 0.11221609 0.2376387 ]
Objective function value at optimal weights: 4.892108513323724e-06
Success: True
Message: Optimization terminated successfully


In [6]:
#METHOD-2 IMPLEMENTATION
w = np.ones(5)/5   #initial guess for weight matrix
C = np.array(C)
def obj_function(w, C):
    return np.sqrt(w.T @ C @ w)


def objective(w):
    return obj_function(w, C)

# Constraints
constraints = (
 {'type': 'ineq', 'fun': lambda w: np.sum(np.log(w)) + 2 },  #I have taken c = -2, you can take other arbitrary constant which gives a feasible soln
    {'type': 'ineq', 'fun': lambda w: w}             # Inequality constraint: w >= 0
)

# Bounds
bounds = [(0, 1) for _ in range(C.shape[1])]



# Optimize
result = minimize(objective, w, method='SLSQP', constraints=constraints, bounds=bounds, options={'disp': True})

# Optimal weights
optimal_weights = result.x

print("Optimal weights:", optimal_weights)
print("Objective function value at optimal weights:", result.fun)
print("Success:", result.success)
print("Message:", result.message)

y = np.array(optimal_weights)
#Normalise
w = y/np.sum(y)
print(w)   #Final weight matrix

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.01503111589719965
            Iterations: 16
            Function evaluations: 97
            Gradient evaluations: 16
Optimal weights: [0.66347187 0.81256351 0.94902347 0.37786066 0.70003931]
Objective function value at optimal weights: 0.01503111589719965
Success: True
Message: Optimization terminated successfully
[0.18940327 0.23196491 0.27092053 0.107869   0.19984229]
